In [1]:
from test_nn import test_model
from lp import run_lp
from os import path
from spectrum_analysis import *
from utils import save_perturbed_test_groups, load_perturbed_test_groups
from utils import load_suspicious_neurons, save_suspicious_neurons
from utils import create_experiment_dir, get_trainable_layers
from utils import load_classifications, save_classifications
from utils import save_layer_outs, load_layer_outs, construct_spectrum_matrices
from utils import load_MNIST, load_CIFAR, load_model
from utils import filter_val_set, save_original_inputs
from input_synthesis import synthesize
from sklearn.model_selection import train_test_split
import datetime
import argparse
import random
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from keras import models

Using TensorFlow backend.


In [2]:
experiment_path = "experiment_results"
model_path = "neural_networks"
group_index = 1
__version__ = "v1.0"


nb_classes = 10
perturbed_count = 10

np.random.seed(42)

In [3]:
args =  {
    'model': 'lenet1',
    'suspicious_num': 10
}

args = defaultdict(lambda: None, args)

In [4]:
model_name     = args['model']
dataset        = args['dataset'] if not args['dataset'] == None else 'mnist'
step_size      = args['step_size'] if not args['step_size'] == None else 1
distance       = args['distance'] if not args['distance'] ==None else 0.1
approach       = args['approach'] if not args['approach'] == None else 'tarantula'
susp_num       = args['suspicious_num'] if not args['suspicious_num'] == None else 1
repeat         = args['repeat'] if not args['repeat'] == None else 1
seed           = args['seed'] if not args['seed'] == None else random.randint(0,10)
star           = args['star'] if not args['star'] == None else 3
logfile_name   = args['logfile'] if not args['logfile'] == None else 'result.log'

In [5]:
####################
# 0) Load MNIST or CIFAR10 data
if dataset == 'mnist':
    X_train, Y_train, X_test, Y_test = load_MNIST(channel_first=False)
else:
    X_train, Y_train, X_test, Y_test = load_CIFAR()

In [6]:
####################
# 1) Load the pretrained network.
try:
    model = load_model(path.join(model_path, model_name))
except:
    logfile.write("Model not found! Provide a pre-trained model model as input.")
    exit(1)

('Model structure loaded from ', 'neural_networks/lenet1')


In [7]:
initial_test_scores = model.evaluate(X_test, Y_test)
initial_train_scores = model.evaluate(X_train, Y_train)
print(initial_train_scores)
print(initial_test_scores)

60000/60000 [==============================] - 39s 645us/step
[0.07679555923460672, 0.97695]
[0.0700444159341976, 0.9788]


In [8]:
def exp(selected_class, X, Y):

    logfile = open(logfile_name, 'a')

    #Fault localization is done per class.
    X, Y = filter_val_set(selected_class, X, Y)

    ####################
    # 2)test the model and receive the indexes of correct and incorrect classifications
    # Also provide output of each neuron in each layer for test input x.
    correct_classifications, misclassifications, layer_outs, predictions = test_model(model, X, Y)

    ####################
    # 3) Receive the correct classifications  & misclassifications and identify 
    # the suspicious neurons per layer
    trainable_layers = get_trainable_layers(model)
    scores, num_cf, num_uf, num_cs, num_us = construct_spectrum_matrices(model,
                                                                        trainable_layers,
                                                                        correct_classifications,
                                                                        misclassifications,
                                                                        layer_outs)

    filename = experiment_path + '/' + model_name + '_C' + str(selected_class) + '_' +\
    approach +  '_SN' +  str(susp_num)

    if approach == 'tarantula':
        suspicious_neuron_idx = tarantula_analysis(trainable_layers, scores,
                                             num_cf, num_uf, num_cs, num_us,
                                             susp_num)
    else:
        print('Wrong approach')
        exit()


    logfile.write('Suspicous neurons: ' + str(suspicious_neuron_idx) + '\n')

    ####################
    # 4) Run Suspiciousness-Guided Input Synthesis Algorithm
    # Receive the set of suspicious neurons for each layer from Step 3 # and 
    # will produce new inputs based on the correct classifications (from the 
    # testing set) that exercise the suspicious neurons

    perturbed_xs = []
    perturbed_ys = []

    # select 10 inputs randomly from the correct classification set.
    #selected = list(correct_classifications)
    if perturbed_count == -1:
        selected = list(correct_classifications)
    else:
        selected = np.random.choice(list(correct_classifications), size=perturbed_count, replace=False)
        
    zipped_data = zip(list(np.array(X)[selected]), list(np.array(Y)[selected]))

    syn_start = datetime.datetime.now()
    x_perturbed, y_perturbed, x_original = synthesize(model, zipped_data,
                                           suspicious_neuron_idx,
                                           correct_classifications,
                                           step_size,
                                           distance)
    syn_end = datetime.datetime.now()

    perturbed_xs = perturbed_xs + x_perturbed
    perturbed_ys = perturbed_ys + y_perturbed

    # reshape them into the expected format
    perturbed_xs = np.asarray(perturbed_xs).reshape(np.asarray(perturbed_xs).shape[0],
                                     *X[0].shape)
 
    perturbed_ys = np.asarray(perturbed_ys).reshape(np.asarray(perturbed_ys).shape[0], 10)

    for i in range(len(perturbed_xs)):
        name = 'susp_adv_inputs/'+model_name+'_'+str(perturbed_ys[i])+'_C'+str(selected_class)+'_susp_guided_'+str(selected[i])+'.png'
        plt.imsave(name, perturbed_xs[i].reshape(28,28), cmap='gray')

    ####################
    # 5) Test if the mutated inputs are adversarial
    score = model.evaluate(perturbed_xs, perturbed_ys, verbose=0)
    logfile.write('Model: ' + model_name + ', Class: ' + str(selected_class) +
                  ', Approach: ' + approach + ', Distance: ' +
                  str(distance) + ', Score: ' + str(score) + '\n')

    logfile.write('Input Synthesis Time: ' + str(syn_end-syn_start) + '\n')

    logfile.close()
    
    return perturbed_xs, perturbed_ys, suspicious_neuron_idx

In [9]:
perturbed_xs_by_class, perturbed_ys_by_class, sus_ids_by_class = range(nb_classes), range(nb_classes), range(nb_classes)

for selected_class in range(nb_classes):
    perturbed_xs_by_class[selected_class], perturbed_ys_by_class[selected_class], sus_ids_by_class[selected_class] = exp(selected_class, X_train, Y_train)

Validation set filtered for desired class: 0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 28, 28, 4)         104       
_________________________________________________________________
block1_pool1 (MaxPooling2D)  (None, 14, 14, 4)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 14, 14, 12)        1212      
_________________________________________________________________
block2_pool1 (MaxPooling2D)  (None, 7, 7, 12)          0         
_________________________________________________________________
flatten (Flatten)            (None, 588)               0         
_________________________________________________________________
softmax (Dense)              (N

/anaconda3/envs/tensorflow_env_p27/lib/python2.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5923
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0

   micro avg       0.99      0.99      0.99      5923
   macro avg       0.10      0.10      0.10      5923
weighted avg       1.00      0.99      1.00      5923

[[5865    2   13    4    5    9   14    3    4    4]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0   

('Input index:', 5)
('Input index:', 10)
Validation set filtered for desired class: 4
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 28, 28, 4)         104       
_________________________________________________________________
block1_pool1 (MaxPooling2D)  (None, 14, 14, 4)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 14, 14, 12)        1212      
_________________________________________________________________
block2_pool1 (MaxPooling2D)  (None, 7, 7, 12)          0         
_________________________________________________________________
flatten (Flatten)            (None, 588)               0         
________________________________________________________

('Input index:', 10)
Validation set filtered for desired class: 7
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 28, 28, 4)         104       
_________________________________________________________________
block1_pool1 (MaxPooling2D)  (None, 14, 14, 4)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 14, 14, 12)        1212      
_________________________________________________________________
block2_pool1 (MaxPooling2D)  (None, 7, 7, 12)          0         
_________________________________________________________________
flatten (Flatten)            (None, 588)               0         
_________________________________________________________________
softmax (D

('Input index:', 5)
('Input index:', 10)


In [10]:
a=np.concatenate(perturbed_xs_by_class)
a=np.concatenate([a, X_train])
b=np.concatenate(perturbed_ys_by_class)
b=np.concatenate([b, Y_train])

In [11]:
print(initial_train_scores)
print(initial_test_scores)

[0.07679555923460672, 0.97695]
[0.0700444159341976, 0.9788]


In [12]:
new_model = models.model_from_json(model.to_json())
new_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
new_hist = new_model.fit(a, b, validation_data=(X_test, Y_test), epochs=5, batch_size=32)

cont_hist = model.fit(a, b, validation_data=(X_test, Y_test), epochs=5, batch_size=32)

Train on 60100 samples, validate on 10000 samples
Epoch 1/5
60100/60100 [==============================] - 98s 2ms/step - loss: 0.2337 - acc: 0.9305 - val_loss: 0.0923 - val_acc: 0.9692
Epoch 2/5
60100/60100 [==============================] - 93s 2ms/step - loss: 0.0858 - acc: 0.9739 - val_loss: 0.0627 - val_acc: 0.9792
Epoch 3/5
60100/60100 [==============================] - 103s 2ms/step - loss: 0.0653 - acc: 0.9798 - val_loss: 0.0503 - val_acc: 0.9850
Epoch 4/5
60100/60100 [==============================] - 102s 2ms/step - loss: 0.0522 - acc: 0.9838 - val_loss: 0.0533 - val_acc: 0.9826
Epoch 5/5
60100/60100 [==============================] - 99s 2ms/step - loss: 0.0448 - acc: 0.9854 - val_loss: 0.0476 - val_acc: 0.9840
Train on 60100 samples, validate on 10000 samples
Epoch 1/5
60100/60100 [==============================] - 99s 2ms/step - loss: 0.0839 - acc: 0.9738 - val_loss: 0.0593 - val_acc: 0.9808
Epoch 2/5
60100/60100 [==============================] - 100s 2ms/step - loss: 0.0

In [13]:
retrained_metrics = new_model.evaluate(X_test, Y_test)
print(retrained_metrics)

10000/10000 [==============================] - 7s 667us/step
[0.04762123263997491, 0.984]


In [14]:
model.evaluate(X_train, Y_train)

60000/60000 [==============================] - 43s 717us/step


[0.030830170396583464, 0.99055]

In [15]:
model.evaluate(X_test, Y_test)

10000/10000 [==============================] - 7s 689us/step


[0.04019824885940179, 0.9865]

In [16]:
#np.save('perturbed_xs_by_class', perturbed_xs_by_class)
#np.save('perturbed_ys_by_class', perturbed_ys_by_class)